In this notebook, I will be reproducing the experimental results in "Recommendations as Treatments: Debiasing Learning and Evaluation". 

# Premise
## Why debiase learning and evaluation in recommender systems?
Recommender systems present sparse data that is prone to selection biases as users are more likely to rate items that they like or know. This means data is MNAR (Missing Not At Random) as observations are conditioned on what we like to optimize (star rating indicating quality of recommendation). MNAR data is a problem as it distorts evaluating rating and recommendation quality estimators. 

## Framing the problem
This paper frames recommender systems as treatments in medical system. Precisely, recommending an item to a user is analogous to giving a new treatment to a patient. This allows us to view recommendars in casual inference setting and 
to derive a framework for unbiased evaluation and learning with MNAR data. 

## Contribution of the paper
First, the authors demonstrate propensity-weighing techniques to develop unbiased rating and recommendation quality estimators.
Secondly, the authors propose Empirical Risk Minimization (ERM) framework and derive a matrix factorization model that can learn under the presence of selection bias in data.
Last, the authors explore techniques for estimating propensities in observation settings and demonstrate the robustness of the framework when propensities are misspecified.  

# Steps to reproduce
6.2) How does sampling bias severity affect evaluation?
- Step 1: Prepare synthetic ML100K dataset
- Step 2: Prepare ML100K Observation Models
- Step 3: Illustrate the effectiveness of unbiased estimators on semi-synthetic dataset and observation models

6.3) How does samplig bias severity affect learning?
- Step 4: Compare MF-IPS and MF-Naive models

6.4) How robust is evaluation and learning to inaccurately learned propensities?
- Step 5: Create propensity estimation models
- Step 6: Demonstrate performances of propensity estimators and matrix factorizations under observational settings

6.5) How do estimators perform on real data?
- Step 7: Demonstrate performances of propensity estimators and matrix factorizations with real data 

## Repository structure in /code

In [18]:
# /generated : pickle files that store data between steps to save duplicate computations
# /library 
    # /estimators
        # /ips.py
        # /naive.py
        # /snips.py
    # /factorization
        # /mf_ips.py
        # /mf_naive.py
    # /util
# /rawdata
    # /coat
    # /ml-100k
# step1.py
# step2.py
# ..
# step7.py

## Libraries Used

In [1]:
import pandas as pd
import numpy as np
import pickle

# How does sampling bias affect evaluation?
## Step 1) Prepare ML100K semi-synthetic dataset 

In ML100K dataset, there are 100K ratings for 1683 movies by 944 users. This means the matrix has ~6% of entries filled. To enable ground-truth evaluation against a complete dataset, we synthetically fill in missing entries as demonstrated below.

### View underlying data
We import raw data (u.data) file in ML100K dataset that contains MNAR ratings for 1683 movies by 944 users.

In [2]:
# snippet in code/step1.py

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('rawdata/ml-100k/u.data', sep='\t', names=r_cols,
                      encoding='latin-1', usecols=[0,1,2])

ratings_df = ratings.pivot(index='user_id', columns='movie_id', values='rating')
ratings_df[:10] # show top 10

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,4.0,NaN,NaN,NaN,NaN,NaN,2.0,4.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,5.0,NaN,NaN,5.0,5.0,5.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,5.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# View sparsity 
print "Sparsity : ", sum(ratings_df.notnull().sum(axis=1).tolist()) \
                / float((ratings_df.shape[1] * ratings_df.shape[0])) * 100, "%"

Sparsity :  6.30466936422 %


In [4]:
# View marginal probabilities of ratings
ratings = ratings_df.fillna(0).values
hist, binEdges = np.histogram(np.clip(ratings, 0, 5), bins = range(1,7), density = False)
print hist

[ 6110 11370 27145 34174 21201]


### Designing Matrix Factorization model
To fill in missing values, we use standard matrix factorization with propensities weighing. We assume rank d-restricted and L2-regularized matrix factorization model with the below training objective (Eq. 14 in paper)

In [5]:
%%latex 
\begin{eqnarray*} argmin(V,W,A) & [ \sum_{O_{u,i}=1} \frac{\delta_{u,i} (Y, V^{T}W + A)}{P_{u,i}} & 
                    + \gamma(||V||^{2}_{F} + ||W||^{2}_{F})] \end{eqnarray*}

<IPython.core.display.Latex object>

This objective is implemented below.

In [23]:
def objective(startVector):
    """
    argmin(V,W,A) ( sum of normalized errors (by propensities) between Y and V.T * W + A
                    + Reg * (||V||^2F + ||W||^2F)
                    V = user matrix
                    W = item matrixp
                    A = offset terms; user, item, global offsets
    """
    userVectors, itemVectors, userBiases, itemBiases, globalBias = reconstruct2Darrays(startVector)

    currentPrediction = np.dot(userVectors, itemVectors.T)
    delta = np.subtract(ratings, currentPrediction)
    loss = np.square(delta)

    normalizedLoss = np.multiply(loss, inversePropensities)  #inversePropensities is defined outside func
    objective = np.ma.sum(normalizedLoss, dtype=np.longdouble)

    objective += reg * np.sum(np.square(userBiases), dtype = np.longdouble)
    objective += reg * np.sum(np.square(itemBiases), dtype = np.longdouble)
    objective += reg * np.sum(np.square(globalBias))

    return objective

The paper states using LBFGS solver to minimize this objective. scipy.optimize.minimize library was chosen as an optimization framework as it provides an implementation of LBFGS solver. Below is the full implementation of a matrix factorization model using this library. An instance of this model can be created with a training matrix, regularization parameter, number of dimensions (ranks), and propensities. Training the instantiated model (learning userVecs, itemVecs, userBiases, itemBiases, globalBiases vectors) can be done by calling train() on the instance. Then, prediction matrix can be reconstructed by calling predict_all() after training. 

In [6]:
import numpy as np
import scipy

"""
Standard matrix factorization with propensity normalization
http://activisiongamescience.github.io/2016/01/11/Implicit-Recommender-Systems-Biased-Matrix-Factorization/
"""

class MF():
    """
    Train a matrix factorization model to predict empty
    entries in a user x item matrix with rating values
    """
    def __init__(self,
                 ratings,
                 ranks=40,
                 reg=0.0,
                 allPropensities=None,
                 verbose=False):

        # Store Inputs
        self.ratings = ratings # masked array with 0 values masked out
        self.allPropensities = allPropensities
        self.ranks = ranks
        self.reg = reg  # regularization factor
        self.numUsers, self.numItems = self.ratings.shape 

        # start vectors and used throughout optimiation
        self.userVecs = np.random.normal(size=(self.numUsers, self.ranks))  # 943 X 200
        self.itemVecs = np.random.normal(size=(self.numItems, self.ranks))  # 1682 X 200
        self.userBias = np.zeros(self.numUsers)  # 943 x 1 - what average rating does user give to all movies?
        self.itemBias = np.zeros(self.numItems)  # 1682 x 1
        self.globalBias = np.mean(self.ratings[np.where(self.ratings != 0)])
        self.resultVectors = None
    
    def train(self):
        numUsers, numItems = self.numUsers, self.numItems
        rank = self.ranks
        reg = self.reg
        ratings = self.ratings
        allPropensities = self.allPropensities
        invPropensities = np.ma.divide(1.0, allPropensities)
        inversePropensities = np.array(invPropensities, dtype = np.longdouble, copy = True)
        inversePropensities = np.ma.array(inversePropensities, dtype = np.longdouble, copy = False,
                            mask = np.ma.getmask(ratings), fill_value = 0, hard_mask = True)
        inversePropensities = np.ma.filled(inversePropensities, 0.0)
        ratings = np.ma.filled(ratings, 0)
        numUsers, numItems = np.shape(ratings)

        def flatten2Darray(user_vectors, item_vectors, userBiases, itemBiases, globalBias):
            allUserParams = np.concatenate((user_vectors, userBiases[:, None]), axis = 1) # 943 X 201
            allItemParams = np.concatenate((item_vectors, itemBiases[:, None]), axis = 1) # 1682 X 201
            allParams = np.concatenate((allUserParams, allItemParams), axis = 0) # (2625, 201)
            paramVector = np.reshape(allParams, (numUsers + numItems) * (rank + 1)) # (2625, 201), (1682 + 943 * 201), (527626,)
            paramVector = np.concatenate((paramVector, [globalBias]))
            return paramVector.astype(np.float)

        def reconstruct2Darrays(paramVector):
            globalBias = paramVector[-1]
            allParams = np.reshape(paramVector[:-1], (numUsers + numItems, rank + 1))
            allUserParams = allParams[0:numUsers, :]
            allItemParams = allParams[numUsers:, :]
            userVectors = (allUserParams[:, 0:-1]).astype(np.longdouble)
            userBiases = (allUserParams[:, -1]).astype(np.longdouble)
            itemVectors = (allItemParams[:, 0:-1]).astype(np.longdouble)
            itemBiases = (allItemParams[:, -1]).astype(np.longdouble)
            return userVectors, itemVectors, userBiases, itemBiases, globalBias
        
        def objective(startVector):
            """
            argmin(V,W,A) ( sum of normalized errors (by propensities) between Y and V.T * W + A
                            + Reg * (||V||^2F + ||W||^2F)
                            V = user matrix
                            W = item matrixp
                            A = offset terms; user, item, global offsets
            """
            userVectors, itemVectors, userBiases, itemBiases, globalBias = reconstruct2Darrays(startVector)
            
            currentPrediction = np.dot(userVectors, itemVectors.T)
            delta = np.subtract(ratings, currentPrediction)
            loss = np.square(delta)

            normalizedLoss = np.multiply(loss, inversePropensities)
            objective = np.ma.sum(normalizedLoss, dtype=np.longdouble)

            objective += reg * np.sum(np.square(userBiases), dtype = np.longdouble)
            objective += reg * np.sum(np.square(itemBiases), dtype = np.longdouble)
            objective += reg * np.sum(np.square(globalBias))

            return objective

        flattenedStartVector = flatten2Darray(self.userVecs, self.itemVecs, self.userBias, self.itemBias, self.globalBias)

        print "- BEGINNING OPTIMIZATION FOR RANK = " + str(rank) + "AND L2 REG PARAM = " + str(reg) + " -"

        ops = {'maxiter': 50, 'disp': True, 'eps' : 1e0}
        result = scipy.optimize.minimize(fun=objective, x0=flattenedStartVector,  # x0 = initial guess
                        method = 'L-BFGS-B', tol = 1, jac=None, options = ops)

        print "Did optimizer exit successfully? = " + str(result['success'])
        rUV, rIV, rUB, riB, rgB = reconstruct2Darrays(result['x'])

        # store in the object
        self.resultVectors = (rUV, rIV, rUB, riB, rgB)

    def predict_all(self):
        rUV, rIV, rUB, riB, rgB = self.resultVectors
        return np.dot(rUV, rIV.T) + rUB[:,None] + riB[None,:] + rgB


Unfortunately, this factorization model did not converge after a long time. After profiling, we discovered that scipy.optimize.minimize step takes a very long time to converge for this unconstrained problem. For now, we use an alternative model that uses stochastic gradient function as implemented below. 

In [7]:
import numpy as np

class MF():
    """
    Train a matrix factorization model to predict empty
    entries in a user x item matrix with rating values
    """
    def __init__(self,
                 ratings,
                 ranks=40,
                 reg=0.0,
                 allPropensities=None):

        # Store Inputs
        self.ratings = np.ma.array(ratings, mask=ratings <= 0)  # masked array with 0 values masked out
        self.allPropensities = allPropensities
        self.ranks = ranks
        self.reg = reg  # regularization factor
        self.numUsers, self.numItems = self.ratings.shape
        self.validRow, self.validCol = self.ratings.nonzero()
        self.numValidEntries = len(self.validRow)

        # start vectors and used throughout optimization
        self.userVecs = np.random.normal(size=(self.numUsers, self.ranks))  # 943 X 200
        self.itemVecs = np.random.normal(size=(self.numItems, self.ranks))  # 1682 X 200
        self.userBias = np.zeros(self.numUsers)  # 943 x 1 - what average rating does user give to all movies?
        self.itemBias = np.zeros(self.numItems)  # 1682 x 1
        self.globalBias = np.mean(self.ratings[np.where(self.ratings != 0)])
        self.resultVectors = None

    def train(self, numIters, learningRate=1e-4):
        for i in range(0, numIters):
            print "Iteration # = " + str(i)
            self.sgd(learningRate)
        self.resultVectors = (self.userVecs, self.itemVecs, self.userBias, self.itemBias, self.globalBias)

    def sgd(self,learningRate):
        np.random.shuffle(np.arange(self.numValidEntries))

        for indx in np.arange(self.numValidEntries):

            u = self.validRow[indx]
            i = self.validCol[indx]
            prediction = self.predict(u, i)
            e = (self.ratings[u,i] - prediction) # error

            # Update biases
            self.userBias[u] += learningRate * \
                                (e - self.reg * self.userBias[u])
            self.itemBias[i] += learningRate * \
                                (e - self.reg * self.itemBias[i])

            # Update latent factors
            self.userVecs[u, :] += learningRate * \
                                    (e * self.itemVecs[i, :] -
                                     self.reg * self.userVecs[u, :])
            self.itemVecs[i, :] += learningRate * \
                                    (e * self.userVecs[u, :] -
                                     self.reg * self.itemVecs[i, :])

    def predict(self, u, i):
        """ Single user and item prediction."""
        prediction = self.globalBias + self.userBias[u] + self.itemBias[i]
        prediction += self.userVecs[u, :].dot(self.itemVecs[i, :].T)
        return prediction

    def predict_all(self):
        rUV, rIV, rUB, riB, rgB = self.resultVectors
        return np.dot(rUV, rIV.T) + rUB[:,None] + riB[None,:] + rgB

### Choosing hyperparameters and executing model training 

Using this alternative model, parameter optimization for finding the best combination of latent factor and l2 regularization parameter is implemented. The below method produces all models with all combinations specified by grid search.

In [8]:
def generateAllModels(latent_factors, l2_lambdas, train_data, test_data):
    allModels = []

    for latentFactor in latent_factors:
        for l2_lambda in l2_lambdas:
            print "Produce model for latent factor = " + str(latentFactor) + " and l2 lambda = " + str(l2_lambda)
            model = MF(train_data, ranks=latentFactor, reg=l2_lambda, allPropensities=allPropensities, verbose=True)
            model.train(200) # train with 200 iterations
            predictions = model.predict_all()
            allModels.append((model, predictions))

After producing all models, we iterate through and adjust the predicted matrices of the models in order to better match a more realistic distribution [p1, p2, p3, p4, p5] for ratings 1 to 5 (explained in section 6.2). More specifically, we sort the matrices by their scores and assign the bottom p1 fractions of the entries a rating of 1, the next p2 fraction a rating of 2, and so on. After adjusting, we choose the best model by maximizing the 0/1 accuracy of the completed matrix on the test set. Below is a implementation of a method that goes through all generated models, adjusts the predicted matrices, and picks the best model by calculating 0/1 accuracy.

In [10]:
from library.util import error_measures

def obtainBestModel(allModels, train_data, test_data):
    # Pick the best model whose adjusted matrix produces the best accuracy error
    numRows, numCols = ratings.shape
    numEntries = numRows * numCols

    # Adjust values to better follow marginal distribution given in @Zemel paper
    marginalProbabilities = [0.0611, 0.11370, 0.27145, 0.34174, 0.21201]
    cumulativeProbabilities = np.cumsum(marginalProbabilities, dtype=np.longdouble)
    cumulativeProbabilities = np.insert(cumulativeProbabilities, 0, 0)  # append 0 in the beginning

    bestModel = None
    bestModelScore = -10000
    finalPredictionMatrix = None

    for (model, predictions) in allModels:
        print "\nTesting first model for rank = " + str(model.ranks) + " gamma = " + str(model.reg)
        endpoints = []
        sortedPredictions = np.sort(predictions, axis=None)

        # indices of cutoff elements
        for i in range(0, 5):
            ind = int(cumulativeMarginals[i] * numEntries)
            endpoints.append(sortedPredictions[ind])

        # attach last element of the array
        endpoints.append(sortedPredictions[numEntries - 1])

        print "ENDPOINTS "
        print endpoints

        # adjust ratings to fall in buckets to better follow marginal probabilities
        adjustedMatrix = np.empty((numRows, numCols), dtype=np.int)
        for i in range(1, 6):
            mask = np.logical_and(predictions >= endpoints[i - 1], predictions <= endpoints[i])
            adjustedMatrix[mask] = i

        # only compare train and test errors in observed entries
        train_mask = train_data != 0
        pred_copy_train = np.around(adjustedMatrix[train_mask])
        train_copy = train_data[train_mask]

        test_mask = test_data != 0
        pred_copy_test = np.around(adjustedMatrix[test_mask])
        test_copy = test_data[test_mask]

        # round all numbers in classification matrix
        train_score = error_measures.get_accuracy(pred_copy_train, train_copy)
        test_score = error_measures.get_accuracy(pred_copy_test, test_copy)

        print "Train accuracy score = " + str(train_score) + "\tTesting accuracy score = " + str(test_score)

        if test_score > bestModelScore:
            bestModel = model
            bestModelScore = test_score
            finalPredictionMatrix = adjustedMatrix

    print "\n\n Best model is with rank = " + str(bestModel.ranks) + " and gamma = " + str(bestModel.reg) \
                + "with score" + bestModelScore
    print "--------------- HISTOGRAMS ---------------"
    print np.histogram(finalPredictionMatrix, bins=range(1, 7), density=False)
    print np.histogram(finalPredictionMatrix, bins=range(1, 7), density=True)
    
    return (bestModel, finalPredictionMatrix)

Now, we demonstrate calling the above two defined methods for generating all models and finding the best model. An additional method for producing 90/10 split is implemented. We train the model on a training set and pick the best model by calculating the best 0/1 accuracy score on the test set.

In [11]:
import os 

def train_test_split(ratings):
    """
    produce a 90/10 train test split and assert they are disjoint
    """
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for user in xrange(ratings.shape[0]):
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0],
                                        size=10,
                                        replace=False)
        train[user, test_ratings] = 0.
        test[user, test_ratings] = ratings[user, test_ratings]
        
    # unknown values are filled with 0
    train = np.ma.array(train, mask = train <= 0, fill_value = 0, hard_mask = True, dtype=np.int, copy=False)
    test = np.ma.array(test, mask = test <= 0, fill_value = 0, hard_mask = True, dtype=np.int, copy=False) 

    # Test and training are truly disjoint
    assert(np.all((np.ma.filled(train, fill_value=0) * np.ma.filled(test, fill_value=0)) == 0))
    return train, test


r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('rawdata/ml-100k/u.data', sep='\t', names=r_cols,
                      encoding='latin-1', usecols=[0,1,2])

ratings_df = ratings.pivot(index='user_id', columns='movie_id', values='rating')
ratings_df[:10] # show top 10
ratings = ratings_df.fillna(0).values

train_data, test_data = train_test_split(ratings)

# use a pre-generated file with all models 
prevGeneratedAllModelsPath = "generated/step1_allModels_MF_Naive_2.pkl"

allModels = []

# CHECK IF ALL MODEL FILES HAVE BEEN GENERATED
if os.path.exists(prevGeneratedAllModelsPath):
    g = open(prevGeneratedAllModelsPath, 'rb')
    allModels = pickle.load(g)
    g.close()
else:
    latent_factors = [200]
    l2_lambdas = [1, 5, 25, 125]
    allModels = generateAllModels(latent_factors, l2_lambdas, train_data, test_data)

(bestModel, ML100K_syntheticMatrix) = obtainBestModel(allModels, train_data, test_data)

ML100K_syntheticMatrix

Testing first model for rank = 2 gamma = 0.0001
CHECK ENDPOINTS
[-6.2124959620452334, 2.1988804979567456, 2.8114930751641776, 3.3828218955113751, 4.0191505873334412, 15.124152440025847]
Train accuracy score = 37525Testing accuracy score = 3593
Testing first model for rank = 2 gamma = 0.001
CHECK ENDPOINTS
[-5.8160408092320921, 2.2497007396749971, 2.8260314516827507, 3.3772764900671914, 4.003684531830582, 13.889093499300042]
Train accuracy score = 37561Testing accuracy score = 3597
Testing first model for rank = 2 gamma = 0.01
CHECK ENDPOINTS
[-9.7018731359211277, 2.2110179072657141, 2.8080677090392427, 3.3831453319001428, 4.0173946795295956, 15.081875612655688]
Train accuracy score = 37588Testing accuracy score = 3565
Testing first model for rank = 2 gamma = 0.1
CHECK ENDPOINTS
[-8.3845810253552298, 2.3574725521937951, 2.8949915159335982, 3.3978735967200113, 3.958828045111948, 12.993049626124014]
Train accuracy score = 37667Testing accuracy score = 3601
Testing first model for rank = 2

array([[4, 4, 3, ..., 3, 4, 4],
       [4, 4, 4, ..., 1, 5, 3],
       [4, 3, 3, ..., 5, 1, 5],
       ..., 
       [4, 4, 3, ..., 2, 5, 5],
       [4, 4, 4, ..., 5, 4, 2],
       [4, 3, 4, ..., 4, 3, 3]])

## Step 2) Prepare ML100K observation models

Now that we have a synthetic data set, we can create different observation models for varying sampling bias. Comparison amongst these models will allow us to observe how unbiased and biased performance estimators behave with increasing level of sampling bias. Unbiased estimators incorporate propensities while biased estimators do not. We will observe whether unbiased estimators produce values that are closer to values of true performance than biased estimators

Different observation models are created by varying values of propensities and marginal probabilities of ratings. We experimentally control the propensity by the below rules. 
    - propensity = k if rating = 4 & 5
    - propensity = k* alpha^(4-r) for rating r < 4
    - for each alpha, set k so that the expected # ratings = 5% of the entire matrix
    
Marginal probabilities and propensities are related by Naive Bayes equation below. 

In [12]:
%%latex
\begin{eqnarray*} (1)  P(O_{u,i} = 1| Y_{u,i} = r) = \frac{ (2)  p(Y = r | O = 1) \cdot (3)  P(O = 1)}{ (4)  p(Y = r)}
\end{eqnarray*}

<IPython.core.display.Latex object>

Term (1) describes propensities of ratings and (2) describes marginal probabiliteis of ratings. Term (3) and (4) can be estimated by counting the number of observed values and rankings. Calculations for propensities are implemented below.

In [42]:
import numpy as np
import sys

"""
    In all methods, we assume iid data which allows us to work with naive bayes equation.
    Prop(Ou,i = 1 | Yu,i = r) = Q(Y = r | O = 1) * P(O = 1) /
                                            P (Y = r)
    Prop = propensity
    Q = marginal probability
"""

def generatePropensityPerRating(observedRatings, ratingMarginals):
    """
        Calculate Prop(Ou,i = 1 | Yu,i = r) given an observation matrix and Q(Y = r | O = 1)
    """
    numObservations = np.ma.count(observedRatings)
    numUsers, numItems = np.shape(observedRatings)
    scale = numUsers * numItems

    observedRatings = np.ma.compressed(observedRatings)    
    observationProbability = 1.0 * numObservations / scale
    histogram = np.bincount(observedRatings, minlength = 6)[1:]
    probabilisticHistogram = 1.0 * (histogram) / numObservations

    propensityPerRating = observationProbability * np.divide(probabilisticHistogram, ratingMarginals)
    propensityPerRating = np.clip(propensityPerRating, a_min = 0, a_max = 1)

    return propensityPerRating

def generateMarginalProbabilities(observedRatings, propensityPerRating):
    """
        Calculate Q(Y = r | O = 1) given an observation matrix and Prop(Ou,i = 1 | Yu,i = r)
    """
    allPropensities = generateAllPropensities(propensityPerRating, observedRatings)
    observedRatings = generatePartiallyObservedRatings(observedRatings, allPropensities)
    observedRatings = np.ma.compressed(observedRatings)
    totalRatings = float(len(observedRatings.ravel()))
    marginals = [x / totalRatings for x in np.bincount(observedRatings.ravel(), minlength=6)[1:]]
    return marginals


def generatePropensitiesPerRatingWithAlpha(completeMatrix, ratingsMarginal, alpha, sparsity):
    """
        Generate propensity per rating with varying alpha values
        For each alpha, set k so that expected number of rating is 5% of the entire matrix.
        For ratings r < 4, the corresponding propensity is k * a4-r.
        For ratings r = 4 & 5, the propensity is equal to k

    """
    numRatings = 5
    numUsers, numItems = completeMatrix.shape
    ratingDistribution = [x * (numUsers * numItems) for x in ratingsMarginal]

    # Set rating propensities
    ratingPropensities = np.zeros(numRatings, dtype=np.longdouble)
    factor = 1.0
    for i in range(5):
        ratingPropensities[numRatings - i - 1] = factor
        if i != 0: factor *= alpha

    # ratingPropensities contain [alpha^3, alpha^2, alpha, 1, 1]]
    #     Prop(Ou,i = 1 | Yu,i = r) = Q(Y = r | O = 1) * P(O = 1) /
    #                                         P (Y = r)
    numObservations = sparsity * numUsers * numItems
    expectedObservations = np.dot(ratingDistribution, ratingPropensities)
    rightSide = numObservations * 1.0 / expectedObservations
    ratingPropensities = np.clip(rightSide * ratingPropensities, a_min = 0, a_max = 1)

    return ratingPropensities

def generateAllPropensities(propensityPerRating, data_matrix, data_disjoint_matrix=None):
    allPropensities = np.zeros(np.shape(data_matrix), dtype=np.longdouble)

    for ind in range(1, 6):
        allPropensities[data_matrix == ind] = propensityPerRating[ind-1]
        if data_disjoint_matrix is not None:
            allPropensities[data_disjoint_matrix == ind] = propensityPerRating[ind-1]

    allPropensities = np.ma.array(allPropensities, dtype=np.longdouble, copy=False,
                        mask=allPropensities <= 0, fill_value=1, hard_mask=True)

    return allPropensities


def generatePartiallyObservedRatings(complete_matrix, propensities):
    # generates randomly observed ratings with propensities
    numUsers, numItems = np.shape(complete_matrix)
    randomllyGeneratedRatings = np.random.random((numUsers, numItems))
    observedRatings = np.ma.array(complete_matrix, dtype=np.int, copy=True,
                            mask=randomllyGeneratedRatings >= propensities, fill_value=0, hard_mask = True)
    return observedRatings

Now we use above methods to see how we can vary observation models with varying degrees of selection bias. 

In [14]:
import pickle
from library.util.propensity import *

"""
Generate observation models for varying degrees of alpha

"""
g = open("generated/step1_bestModel_Naive_2.pkl", 'rb')
(bestModel, ML100K_syntheticMatrix) = pickle.load(g)
g.close()

# generate observation models for varying alphas
alphas = [0, 0.25, 0.5, 0.75, 1.00]
ratingsMarginal = [0.0611, 0.11370, 0.27145, 0.34174, 0.21201]
observationModels = []
for alpha in alphas:
    propensitiesPerRating = generatePropensitiesPerRatingWithAlpha(ML100K_syntheticMatrix, ratingsMarginal, alpha, 0.05)
    marginalProps = generateMarginalProbabilities(ML100K_syntheticMatrix, propensitiesPerRating)
    allPropensities = generateAllPropensities(propensitiesPerRating, ML100K_syntheticMatrix)
    observedRatings = generatePartiallyObservedRatings(ML100K_syntheticMatrix, allPropensities)
    dict = {"alpha": alpha, "propsPerRating": propensitiesPerRating,
            "marginalProbs": marginalProps, "observedRatings": observedRatings}
    observationModels.append(dict)

    print "Marginal Probabilities for alpha = " + str(alpha)
    print marginalProps, "\n"

g = open("generated/step2_observationModels_Naive_2.pkl", 'wb')
pickle.dump(observationModels, g, -1)
g.close()

Marginal Probabilities for alpha = 0
[0.0, 0.0, 0.0, 0.61726815550460457, 0.38273184449539543] 

Marginal Probabilities for alpha = 0.25
[0.0018834058044291638, 0.011199312367276773, 0.10712660531904136, 0.54330569319445854, 0.33648498331479421] 

Marginal Probabilities for alpha = 0.5
[0.011258034661561672, 0.038781900934366162, 0.18746909745553203, 0.47249515067763731, 0.28999581627090282] 

Marginal Probabilities for alpha = 0.75
[0.030796370967741935, 0.076953124999999997, 0.23907510080645161, 0.40287298387096776, 0.25030241935483871] 

Marginal Probabilities for alpha = 1.0
[0.062048807529008605, 0.11440918196364003, 0.27243521096594558, 0.33937233064140199, 0.21173446890000377] 



# Step 3: Illustrate the effectiveness of unbiased estimators on semi-synthetic dataset and observation models


Now, we compare how the estimators perform against these observation models. Below is my implementation of estimators.
Among these, IPS and SNIPS estimators incorporate propensities while naive estimator does not.

In [15]:
def estimate_naive(pred, actual, func='mse'):
    """
    Naive prediction accuracy estimator for estimation
    between a prediction matrix and a partially complete true ratings matrix

    It uses average error over observed entries:
    Rnaive (Ypred) = (1 / # Observations) * sum of errors over observed entries

    """
    pred = np.ma.compressed(pred[np.invert(actual.mask)])
    actual = np.ma.compressed(actual)
    delta = np.ma.subtract(pred, actual)  # subtraction applied for known entries only

    if func == 'mse':
        delta = np.square(delta)
    elif func == 'mae':
        delta = np.ma.abs(delta)
    elif func == 'rmse':
        delta = np.sqrt(np.square(delta))

    sum_errors = np.ma.sum(delta, dtype=np.longdouble)
    score = sum_errors / (len(pred))

    return score

def estimate_ips(pred, actual, propsPerRating, func='mae'):
    """
    Inverse-Propensity-Scoring estimator
    """
    I, U = pred.shape

    pred = np.ma.compressed(pred[np.invert(actual.mask)])
    actual = np.ma.compressed(actual)
    delta = np.ma.subtract(pred, actual)  # subtraction applied for known entries only

    allPropensities = generateAllPropensities(propsPerRating, actual)

    if func == 'mse':
        delta = np.square(delta)
    elif func == 'mae':
        delta = np.ma.abs(delta)
    elif func == 'rmse':
        delta = np.sqrt(np.square(delta))

    errors = np.ma.multiply(delta, np.reciprocal(allPropensities))
    sum_errors = np.ma.sum(errors, dtype=np.longdouble)
    score = sum_errors / (U * I)

    return score

def estimate_snips(pred, actual, propsPerRating, func='mse'):
    """
    Self-Normalized-Inverse-Scoring-Estimator
    """
    # for all known entries, calculate delta
    pred = np.ma.compressed(pred[np.invert(actual.mask)])
    actual = np.ma.compressed(actual)

    delta = np.ma.subtract(pred, actual)
    allPropensities = generateAllPropensities(propsPerRating, actual)

    if func == 'mse':
        delta = np.square(delta)
    elif func == 'mae':
        delta = np.ma.abs(delta)
    elif func == 'rmse':
        delta = np.sqrt(np.square(delta))

    errors = np.ma.multiply(delta, np.reciprocal(allPropensities))
    sum_errors = np.ma.sum(errors, dtype=np.longdouble)
    sum_normalizers = np.ma.sum(np.reciprocal(allPropensities), dtype=np.longdouble)
    score = sum_errors / sum_normalizers

    return score

To measure how these estimators perform with different observation models on various prediction matrices, we curate 4 different prediction matrices. 

In [24]:
def createPredictionMatrices(data_ML100K_synthetic):
    recOnes = createRecOneMatrix(data_ML100K_synthetic)
    recFours = createRecFourMatrix(data_ML100K_synthetic)
    rotate = createRotateMatrix(data_ML100K_synthetic)
    coarsened = createCoarsenedMatrix(data_ML100K_synthetic)
    # skewed = createSkewedMatrix(data_ML100K_synthetic)
    return recOnes, recFours, rotate, coarsened
    

def createRecOneMatrix(actual):
    predictedRatings = actual.copy()
    numFiveStars = (actual == 5).sum()

    oneIndxRows, oneIndxCols = np.where(actual == 1)
    numOneStars = np.shape(oneIndxRows)[0]
    numFlippedRatings = min(numFiveStars, numOneStars)

    randomSubset = np.random.choice(numOneStars, size=numFlippedRatings, replace=False)
    flipIndices = (oneIndxRows[randomSubset], oneIndxCols[randomSubset])

    predictedRatings[flipIndices] = 5

    print "REC_ONES", \
        "Actual \t", np.bincount(actual.ravel(), minlength = 6)[1:],\
        "Modified \t ", np.bincount(predictedRatings.ravel(), minlength = 6)[1:]

    return predictedRatings


def createRecFourMatrix(actual):
    predictedRatings = actual.copy()
    numFiveStars = (actual == 5).sum()

    fourIndxRows, fourIndxCols = np.where(actual == 4)
    numFourStars = np.shape(fourIndxRows)[0]
    numFlippedRatings = min(numFiveStars, numFourStars)

    randomSubset = np.random.choice(numFourStars, size = numFlippedRatings, replace = False)
    flipIndices = (fourIndxRows[randomSubset], fourIndxCols[randomSubset])

    predictedRatings[flipIndices] = 5

    print "REC_FOURS", \
        "Actual \t", np.bincount(actual.ravel(), minlength = 6)[1:],\
        "Modified \t ", np.bincount(predictedRatings.ravel(), minlength = 6)[1:]

    return predictedRatings


def createRotateMatrix(actual):
    predictedRatings = actual.copy()
    predictedRatings = predictedRatings - 1
    predictedRatings[predictedRatings == 0] = 5

    print "ROTATE", \
        "Actual \t\t", np.bincount(actual.ravel(), minlength = 6)[1:],\
        "Modified \t ", np.bincount(predictedRatings.ravel(), minlength = 6)[1:]

    return predictedRatings


def createCoarsenedMatrix(actual):
    predictedRatings = actual.copy()
    predictedRatings[predictedRatings <= 3] = 3
    predictedRatings[predictedRatings >= 4] = 4

    print "COARSENED", \
        "Actual \t", np.bincount(actual.ravel(), minlength = 6)[1:],\
        "Modified \t ", np.bincount(predictedRatings.ravel(), minlength = 6)[1:]

    return predictedRatings


In [25]:
# Create prediction matrices
g = open("generated/step1_bestModel_naive_2.pkl", 'rb')
bestModel, data_ML100K_complete = pickle.load(g)
g.close()

Y_Rec_Ones, Y_Rec_Fours, Y_Rotate, Y_Coarsened = createPredictionMatrices(data_ML100K_complete)

REC_ONES Actual 	[ 96912 180342 430554 542043 336275] Modified 	  [     0 180342 430554 542043 433187]
REC_FOURS Actual 	[ 96912 180342 430554 542043 336275] Modified 	  [ 96912 180342 430554 205768 672550]
ROTATE Actual 		[ 96912 180342 430554 542043 336275] Modified 	  [180342 430554 542043 336275  96912]
COARSENED Actual 	[ 96912 180342 430554 542043 336275] Modified 	  [     0      0 707808 878318      0]


Now with these prediction matrices, we show how estimators perform when alpha = 0.25. We attempt to recreate Table 1. in 3.4 of the paper. 

In [28]:
from library.estimators import ips
from library.estimators import naive
from library.estimators import snips
import pprint

# Create list of prediction matrices
pred_matrices = [('REC_ONES', Y_Rec_Ones),
                 ('REC_FOURS', Y_Rec_Fours),
                 ('ROTATE', Y_Rotate),
                 ('COARSENED', Y_Coarsened)]
                 # ('SKEWED', Y_Skewed),

estimators = ['Truth', 'IPS', 'SNIPS', 'Naive']
measures = ['mae']  # TODO implement DC@50


# load observation models
g = open("generated/step2_observationModels.pkl", 'rb')
observationModels = pickle.load(g)
g.close()

# look up 0.25 in observation models
observationModel25 = (item for item in observationModels if item["alpha"] == 0.25).next()
propensitiesPerRating25 = observationModel25["propsPerRating"]
observedRatings25 = observationModel25["observedRatings"]

step3_data = []
for measure in measures:
    rowCols = []
    for pred in pred_matrices:
        # for estimator in estimators:
        list = []
        for estimator in estimators:
            func = None
            mean = 0

            if measure == 'mae':
                func = error_measures.get_mae
            elif measure == 'mse':
                func = error_measures.get_mse
            
            trueError = func(pred[1], data_ML100K_complete)

            if estimator == 'Truth':
                mean = trueError
            elif estimator == 'IPS':
                mean = ips.estimate_ips(pred[1], observedRatings25, propensitiesPerRating25, func=measure)
            elif estimator == 'SNIPS':
                mean = snips.estimate_snips(pred[1], observedRatings25, propensitiesPerRating25, func=measure)
            elif estimator == 'Naive':
                mean = naive.estimate_naive(pred[1], observedRatings25, func=measure)

            meanAndErr = str(mean)
            list.append(meanAndErr)

        list.insert(0, pred[0])  # attach name
        rowCols.append(list)

    rowCols = np.asarray(rowCols)
    estimators.insert(0, '')
    rowCols = np.asarray(np.vstack((estimators, rowCols)))  # attach labels
    step3_data.append(rowCols)
    estimators.remove('')

step3_data = np.asarray(step3_data)
pp = pprint.PrettyPrinter(width=1000)
pp.pprint(step3_data)

array([[['', 'Truth', 'IPS', 'SNIPS', 'Naive'],
        ['REC_ONES', '0.24439924697', '0.954526567621', '0.955052073817',
         '0.667621595699'],
        ['REC_FOURS', '0.212010269045', '1.16096047483', '1.16159963139',
         '1.05730836215'],
        ['ROTATE', '1.18329943523', '1.37095933681', '1.3717141064',
         '1.42532623236'],
        ['COARSENED', '0.447909560779', '0.851778060192', '0.852246999102',
         '0.717913122492']]], 
      dtype='|S14')


In [ ]:
# PLOT VALUES FOR ALL ALPHAS

# generate observation models for varying alphas
alphas = [0, 0.25, 0.5, 0.75, 1.00]
ratingsMarginal = [0.0611, 0.11370, 0.27145, 0.34174, 0.21201]
observationModels = []
for alpha in alphas:
    propensitiesPerRating = generatePropensitiesPerRatingWithAlpha(ML100K_syntheticMatrix, ratingsMarginal, alpha, 0.05)
    marginalProps = generateMarginalProbabilities(ML100K_syntheticMatrix, propensitiesPerRating)
    allPropensities = generateAllPropensities(propensitiesPerRating, ML100K_syntheticMatrix)
    observedRatings = generateObservedRatings(ML100K_syntheticMatrix, allPropensities)
    dict = {"alpha": alpha, "propsPerRating": propensitiesPerRating, "marginalProbs": marginalProps, "observedRatings": observedRatings}
    observationModels.append(dict)

    print "Marginal Probabilities for alpha = " + str(alpha)
    print marginalProps

# Step 5) Explore how sampling bias severity affects learning (Section 6.3)


# Step 6) Create propensity estimation model for observation setting 

# Step 7) Study performance of matrix-factorization model on real world dataset 